# Movie Recommendation System spaCy

### Installations for Google Colab

In [102]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg
nlp = en_core_web_lg.load()

# https://stackoverflow.com/questions/56927602/unable-to-load-the-spacy-model-en-core-web-lg-on-google-colab
!pip install contractions

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


### Import Libraries and Data

In [103]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import re
import string
import random 
import requests
import contractions

from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn import metrics 

import spacy

from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.models import KeyedVectors

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


%matplotlib inline

pd.set_option("display.max_columns",350)

In [105]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/df_to_preprocess.csv')
df.drop(columns = 'Unnamed: 0', inplace = True)
print(df.shape)
df.head()

(13173, 41)


,rt_id,critic_name,publisher,review_type,review_date,review_content,title,plot,critics_consensus,content_rating,genre,director,writer,cast,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,critic_id,review_score,year,imdb_title,country,imdb_plot,imdb_score,imdb_count,review_negative,review_neutral,review_positive,review_compound,consensus_negative,consensus_neutral,consensus_positive,consensus_compound,score
0,m/0814255,Ben McEachen,Sunday Mail (Australia),1,2010-02-09,Whether audiences will get behind The Lightnin...,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,0,49.0,149.0,0,53.0,254421.0,643,70.0,2010,tt0814255,"UK, Canada, USA",A teenager discovers he's the descendant of a ...,59.0,174198.0,0.080,0.727,0.193,0.7050,0.000,0.590,0.410,0.9274,53.95
1,m/0814255,Nick Schager,Slant Magazine,0,2010-02-10,Harry Potter knockoffs don't come more transpa...,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,0,49.0,149.0,0,53.0,254421.0,4714,25.0,2010,tt0814255,"UK, Canada, USA",A teenager discovers he's the descendant of a ...,59.0,174198.0,0.000,1.000,0.000,0.0000,0.000,0.590,0.410,0.9274,53.95
2,m/0878835,Erik Childress,eFilmCritic.com,1,2010-01-31,Holofcener always gives us more to chew on tha...,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,2012-09-04,90.0,Sony Pictures Classics,1,87.0,142.0,1,64.0,11574.0,2008,75.0,2010,tt0878835,USA,"In New York City, a husband and wife butt head...",66.0,10928.0,0.000,0.928,0.072,0.4767,0.000,0.885,0.115,0.5023,73.53
3,m/0878835,Richard Mowe,Boxoffice Magazine,1,2010-03-24,"The tone is low-key but very funny. Yet, at th...",Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,2012-09-04,90.0,Sony Pictures Classics,1,87.0,142.0,1,64.0,11574.0,5187,60.0,2010,tt0878835,USA,"In New York City, a husband and wife butt head...",66.0,10928.0,0.000,0.816,0.184,0.6474,0.000,0.885,0.115,0.5023,73.53
4,m/10,Scott Weinberg,eFilmCritic.com,1,2002-07-25,Obvious but entertaining portrayal of midlife ...,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,2014-07-24,122.0,Waner Bros.,1,67.0,24.0,0,53.0,14684.0,5593,80.0,1979,tt0078721,USA,A Hollywood composer goes through a mid-life c...,61.0,14946.0,0.486,0.332,0.182,-0.8126,0.262,0.547,0.191,-0.4676,60.03


### Pre-Process for Text Recommender

In [106]:
tokenizer = word_tokenize
stemmer = WordNetLemmatizer()

In [107]:
df.head(1)

,rt_id,critic_name,publisher,review_type,review_date,review_content,title,plot,critics_consensus,content_rating,genre,director,writer,cast,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,critic_id,review_score,year,imdb_title,country,imdb_plot,imdb_score,imdb_count,review_negative,review_neutral,review_positive,review_compound,consensus_negative,consensus_neutral,consensus_positive,consensus_compound,score
0,m/0814255,Ben McEachen,Sunday Mail (Australia),1,2010-02-09,Whether audiences will get behind The Lightnin...,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,0,49.0,149.0,0,53.0,254421.0,643,70.0,2010,tt0814255,"UK, Canada, USA",A teenager discovers he's the descendant of a ...,59.0,174198.0,0.08,0.727,0.193,0.705,0.0,0.59,0.41,0.9274,53.95


In [108]:
# # Text column 
def create_text(x):
    return (''.join(x['title']) + ' ' + ''.join(x['plot']) + ' ' + ''.join(x['critics_consensus']) + ' ' + ''.join(x['review_content'])
            + ' ' + ''.join(x['genre']) + ' ' + ''.join(x['director']) + ' ' + ''.join(x['cast'] + ' ' + ''.join(x['imdb_plot'])))

df['text'] = df.apply(create_text, axis = 1)

df[['text']].head(2)

,text
0,Percy Jackson & the Olympians: The Lightning T...
1,Percy Jackson & the Olympians: The Lightning T...


In [109]:
stopwords = stopwords.words('english')

In [110]:
def clean_text(text):
    # Set lowercase
    text = text.lower()
    
    # Expand contractions
    text = contractions.fix(text)

    # Remove punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    
     # Remove english stopwords
    text = [i for i in text if i not in stopwords]
    text = ' '.join(text)
    
     # Lemmatize
    text = stemmer.lemmatize(text)
    
    return text
    

In [111]:
df['text'] = df['text'].apply(clean_text)

# Spacy



Testing out another Content-Based Filtering Recommender System that uses the library spaCy, which is a library for advanced natural language processing.

This code has been replicated from Betty LD's post in Towards Data Science [[1]](https://towardsdatascience.com/build-a-text-recommendation-system-with-python-e8b95d9f251c)


In [112]:
# Get best index
cosine = {}
def get_idx(matrix, mask = None):
    
    # returns sum of all tokens cosines for each sentence
    if len(matrix.shape) > 1:
      cos_sim = np.mean(matrix, axis=0) 
    else:
      cos_sim = matrix 
    
    # Ranking index from highest to smallest 
    index = np.argsort(cos_sim)[::-1]
    
    # Returning an array of cosine similarity shape, filled with 1s 
    mask = np.ones(len(cos_sim))
    
    # Setting up truth value to ensure arrays are same shape
    mask = np.logical_or(cos_sim[index], mask)
    
    # Assigning index
    best_idx = index[mask][:20]
    
    # Setting dict for cosine to include with dataframe results
    cosine['cosine'] = (cos_sim)
    
    return best_idx

In [113]:
def get_spacy(text, embed_matrix):
    
    # Using nlp model to set up query and return matrix
    query = nlp(text)
    matrix = np.array([query.similarity(line) for line in embed_matrix])

    # If vector has normalization return True, else return False 
    matrix_mask = np.array([True if line.vector_norm else False for line in embed_matrix])
    
    best_idx = get_idx(matrix, mask = matrix_mask)
    return best_idx

# Applying model to text
df['new_text'] = df['text'].apply(lambda x: nlp(x)) 

# Retrieving the embedded vectors as a matrix 
embed_matrix = df['new_text'].values

In [114]:
text = 'Christmas'
best_idx = get_spacy(text, embed_matrix)
cosine = pd.DataFrame(cosine)
new_rec = (df[['title','plot','score','tomatometer_rating','audience_rating','imdb_score']].iloc[best_idx])
new_rec = pd.concat([new_rec, cosine], axis = 1)
new_rec = new_rec.sort_values(by = ['score'], ascending = False)
display(new_rec.drop_duplicates(subset = 'title', keep = 'first')[:10])

,title,plot,score,tomatometer_rating,audience_rating,imdb_score,cosine
8244,The Nightmare Before Christmas,The film follows the misadventures of Jack Ske...,87.68,95.0,91.0,80.0,0.648425
328,A Christmas Story,Based on the humorous writings of author Jean ...,86.92,89.0,88.0,79.0,0.657321
5836,Holiday Inn,"In this Irving Berlin musical, Jim (Bing Crosb...",83.81,100.0,86.0,74.0,0.629757
637,White Christmas,Singers Bob Wallace (Bing Crosby) and Phil Dav...,77.26,77.0,88.0,76.0,0.638068
8125,National Lampoon's Christmas Vacation,"As the holidays approach, Clark Griswold (Chev...",73.70,67.0,86.0,76.0,0.641074
11489,The Man Who Invented Christmas,"In 1843 London, author Charles Dickens finds h...",72.62,79.0,77.0,70.0,0.672386
10973,The Christmas Chronicles,Siblings Kate and Teddy Pierce hatch a scheme ...,68.81,67.0,77.0,71.0,0.705584
5228,Get Santa,"After crashing his sleigh, Santa Claus (Jim Br...",64.08,79.0,53.0,62.0,0.665350
5146,Fun & Fancy Free,Ventriloquist Edgar Bergen introduces cartoons...,61.77,67.0,61.0,66.0,0.632694
11213,The Grinch,"The Grinch and his loyal dog, Max, live a soli...",58.48,60.0,50.0,63.0,0.655138


In [115]:
text = 'I want to watch an action movie with gangsters like the godfather or scarface.' 
best_idx = get_spacy(text, embed_matrix)
cosine = pd.DataFrame(cosine)
new_rec = (df[['title','plot','score','tomatometer_rating','audience_rating','imdb_score']].iloc[best_idx])
new_rec = pd.concat([new_rec, cosine], axis = 1)
new_rec = new_rec.sort_values(by = ['score'], ascending = False)
display(new_rec.drop_duplicates(subset = 'title', keep = 'first')[:10])

,title,plot,score,tomatometer_rating,audience_rating,imdb_score,cosine
3817,Creep 2,A video artist quickly realizes she has made a...,78.15,100.0,72.0,64.0,0.838594
11977,This Is Not a Film,Iranian filmmaker Jafar Panahi makes a video d...,78.13,97.0,69.0,57.0,0.848682
3815,Creep,Aaron answers an online ad and drives to a str...,70.40,89.0,65.0,48.0,0.841842
2094,American Buffalo,Pawn shop owner Don (Dennis Franz) discovers t...,60.14,74.0,46.0,58.0,0.835455
8884,Porno,"Four repressed, religious teens and a straight...",57.95,81.0,34.0,52.0,0.839430
11260,The Hitman's Bodyguard,The world's top protection agent is called upo...,57.92,43.0,67.0,69.0,0.839478
12216,Triple Threat,A crime syndicate places a hit on a billionair...,55.45,69.0,36.0,55.0,0.851708
8267,No Escape,"American businessman Jack Dwyer (Owen Wilson),...",55.41,47.0,62.0,56.0,0.841795
12818,When We First Met,After spending a night with the girl of his dr...,53.97,43.0,57.0,64.0,0.832862
12393,Unfriended: Dark Web,"After finding a laptop, a young man goes onlin...",53.66,59.0,38.0,59.0,0.839151


In [116]:
text = 'Movie about college' 
best_idx = get_spacy(text, embed_matrix)
cosine = pd.DataFrame(cosine)
new_rec = (df[['title','plot','score','tomatometer_rating','audience_rating','imdb_score']].iloc[best_idx])
new_rec = pd.concat([new_rec, cosine], axis = 1)
new_rec = new_rec.sort_values(by = ['score'], ascending = False)
display(new_rec.drop_duplicates(subset = 'title', keep = 'first')[:10])

,title,plot,score,tomatometer_rating,audience_rating,imdb_score,cosine
11229,The Half of It,"A shy, introverted student helps the school jo...",80.95,97.0,80.0,69.0,0.747647
7675,Mid90s,"In 1990s Los Angeles, a 13-year-old spends his...",76.38,80.0,80.0,74.0,0.751700
8919,Premature,On a summer night in Harlem during her last mo...,74.46,92.0,68.0,58.0,0.756728
10052,Snatchers,"After a teen has sex for the first time, she w...",72.69,100.0,60.0,57.0,0.749985
3031,Brad's Status,Brad Sloan has a satisfying career and a comfo...,67.07,79.0,55.0,65.0,0.749557
2526,Barry,Barack Obama arrives in New York in the fall o...,64.43,80.0,52.0,58.0,0.745275
11552,The Nines,"Three actors (Ryan Reynolds, Hope Davis, Melis...",63.02,65.0,61.0,63.0,0.745351
8884,Porno,"Four repressed, religious teens and a straight...",57.95,81.0,34.0,52.0,0.765939
4867,Film Geek,A socially inept cinephile (Melik Malkasian) i...,54.89,64.0,34.0,61.0,0.748965
9736,Sex Drive,Loveable teen Ian (Josh Zuckerman) has always ...,53.48,45.0,58.0,65.0,0.763148


In [117]:
text = 'Martin Scorsese' 
best_idx = get_spacy(text, embed_matrix)
cosine = pd.DataFrame(cosine)
new_rec = (df[['title','plot','score','tomatometer_rating','audience_rating','imdb_score']].iloc[best_idx])
new_rec = pd.concat([new_rec, cosine], axis = 1)
new_rec = new_rec.sort_values(by = ['score'], ascending = False)
display(new_rec.drop_duplicates(subset = 'title', keep = 'first')[:10])

,title,plot,score,tomatometer_rating,audience_rating,imdb_score,cosine
9088,Raging Bull,The story of a middleweight boxer as he rises ...,89.64,93.0,93.0,82.0,0.684813
4111,The Departed,South Boston cop Billy Costigan (Leonardo DiCa...,88.59,91.0,94.0,85.0,0.681773
449,The King of Comedy,Rupert Pupkin (Robert De Niro) is a failure in...,85.83,89.0,90.0,78.0,0.678418
9063,Quiz Show,Queens-born Herbie Stempel (John Turturro) bec...,85.34,96.0,87.0,75.0,0.681573
1091,Traffic,A contemporary thriller set in the world of dr...,84.90,93.0,85.0,81.0,0.680212
3361,Catch Me If You Can,"Frank Abagnale, Jr. (Leonardo DiCaprio) worked...",84.42,96.0,89.0,61.0,0.684281
11896,The Wolf of Wall Street,"In 1987, Jordan Belfort (Leonardo DiCaprio) ta...",81.00,80.0,83.0,82.0,0.696178
1942,The Age of Innocence,Wealthy lawyer Newland Archer (Daniel Day-Lewi...,75.92,84.0,75.0,63.0,0.679203
3707,Confessions of a Dangerous Mind,Game show television producer Chuck Barris (Sa...,74.64,79.0,75.0,70.0,0.682389
3120,Bringing Out the Dead,After a disheartening and haunting career wear...,71.12,72.0,70.0,68.0,0.681410


In [118]:
text = 'Meryl Streep Comedy' 
best_idx = get_spacy(text, embed_matrix)
cosine = pd.DataFrame(cosine)
new_rec = (df[['title','plot','score','tomatometer_rating','audience_rating','imdb_score']].iloc[best_idx])
new_rec = pd.concat([new_rec, cosine], axis = 1)
new_rec = new_rec.sort_values(by = ['score'], ascending = False)
display(new_rec.drop_duplicates(subset = 'title', keep = 'first')[:10])

,title,plot,score,tomatometer_rating,audience_rating,imdb_score,cosine
9018,Pulp Fiction,Vincent Vega (John Travolta) and Jules Winnfie...,92.89,92.0,96.0,89.0,0.584503
5149,Funny Girl,"In this bittersweet, classic musical drama, th...",82.76,93.0,85.0,74.0,0.588066
9765,Shakespeare in Love,"""Shakespeare in Love"" is a romantic comedy for...",82.62,92.0,80.0,71.0,0.585017
10141,Sophie's Choice,"Stingo (Peter MacNicol), a young writer, moves...",78.49,78.0,85.0,76.0,0.594768
11044,The Devil Wears Prada,Andy (Anne Hathaway) is a recent college gradu...,72.77,75.0,76.0,69.0,0.599204
10646,Tallulah,A dissatisfied Beverly Hills housewife tired o...,72.71,85.0,69.0,67.0,0.583163
6553,Julie & Julia,"Frustrated with a soul-killing job, New Yorker...",71.69,77.0,70.0,70.0,0.601897
966,The Impostors,"Two struggling New York actors (Oliver Platt, ...",66.39,62.0,73.0,65.0,0.579952
13125,You're Not You,A life-changing bond develops between a woman ...,61.96,48.0,70.0,73.0,0.582053
8952,Prime,Rafi Gardet (Uma Thurman) is a 37-year-old div...,55.67,50.0,52.0,62.0,0.579894


In [119]:
text = 'I had a long day at work. I want $omething Funny! // or Dumb 23843098^*&^^*^*^' 
best_idx = get_spacy(text, embed_matrix)
cosine = pd.DataFrame(cosine)
new_rec = (df[['title','plot','score','tomatometer_rating','audience_rating','imdb_score']].iloc[best_idx])
new_rec = pd.concat([new_rec, cosine], axis = 1)
new_rec = new_rec.sort_values(by = ['score'], ascending = False)
display(new_rec.drop_duplicates(subset = 'score', keep = 'first')[:10])

,title,plot,score,tomatometer_rating,audience_rating,imdb_score,cosine
3818,Creep 2,A video artist quickly realizes she has made a...,78.15,100.0,72.0,64.0,0.813631
5321,Go Back to China,A spoiled rich girl is sent to work for her fa...,69.37,81.0,71.0,59.0,0.799500
5617,Happy Death Day 2U,Collegian Tree Gelbman wakes up in horror to l...,63.19,71.0,58.0,62.0,0.798624
4653,Every Day,Sixteen-year-old Rhiannon falls in love with A...,59.00,63.0,55.0,57.0,0.800596
4177,Diary of a Wimpy Kid: Dog Days,"It's summertime, and Greg Heffley (Zachary Gor...",58.44,52.0,63.0,63.0,0.798430
4403,Duck Butter,"Two women, jaded by dishonest and broken relat...",54.24,57.0,42.0,53.0,0.805202
12817,When We First Met,After spending a night with the girl of his dr...,53.97,43.0,57.0,64.0,0.802936
12393,Unfriended: Dark Web,"After finding a laptop, a young man goes onlin...",53.66,59.0,38.0,59.0,0.802548
9735,Sex Drive,Loveable teen Ian (Josh Zuckerman) has always ...,53.48,45.0,58.0,48.0,0.793982
12306,Two Night Stand,An online hook-up turns into something more af...,48.61,38.0,45.0,64.0,0.811709


Some of these are working really well and some are not. It appears that when there is a specific name in it or specific word, it works well, but when it's something longer, such as a full sentence, it doesn't do as well. It also has higher cosine values than the conten-based model that used TF-IDF